### Importing the required dependencies

In [1]:
# ignoring warnings
import warnings
warnings.filterwarnings('ignore')

# Importing other required libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# importing required ML libraries
import sklearn
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
import catboost
from xgboost import XGBClassifier

# Checking all the columns 
pd.set_option('display.max_columns',None)

### Loading the data

In [2]:
# Filepaths for train and test dataset
train_dataset=r'C:\Users\DELL\playground-series-s4e12\train.csv'
test_dataset=r'C:\Users\DELL\playground-series-s4e12\test.csv'

In [3]:
# Reading the required training & testing data into a pandas dataframe
df_train=pd.read_csv(train_dataset)
df_test=pd.read_csv(test_dataset)

### Metadata Analysis


In [4]:
# Checking the basic data available
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200000 entries, 0 to 1199999
Data columns (total 21 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   id                    1200000 non-null  int64  
 1   Age                   1181295 non-null  float64
 2   Gender                1200000 non-null  object 
 3   Annual Income         1155051 non-null  float64
 4   Marital Status        1181471 non-null  object 
 5   Number of Dependents  1090328 non-null  float64
 6   Education Level       1200000 non-null  object 
 7   Occupation            841925 non-null   object 
 8   Health Score          1125924 non-null  float64
 9   Location              1200000 non-null  object 
 10  Policy Type           1200000 non-null  object 
 11  Previous Claims       835971 non-null   float64
 12  Vehicle Age           1199994 non-null  float64
 13  Credit Score          1062118 non-null  float64
 14  Insurance Duration    1199999 non-

In [5]:
# Checking the shape
df_train.shape

(1200000, 21)

In [6]:
# Checking the null values
(100*df_train.isnull().sum()/len(df_train))

id                       0.000000
Age                      1.558750
Gender                   0.000000
Annual Income            3.745750
Marital Status           1.544083
Number of Dependents     9.139333
Education Level          0.000000
Occupation              29.839583
Health Score             6.173000
Location                 0.000000
Policy Type              0.000000
Previous Claims         30.335750
Vehicle Age              0.000500
Credit Score            11.490167
Insurance Duration       0.000083
Policy Start Date        0.000000
Customer Feedback        6.485333
Smoking Status           0.000000
Exercise Frequency       0.000000
Property Type            0.000000
Premium Amount           0.000000
dtype: float64

In [7]:
# Checking the first five rows
df_train.head()

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
0,0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,Premium,2.0,17.0,372.0,5.0,2023-12-23 15:21:39.134960,Poor,No,Weekly,House,2869.0
1,1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,Comprehensive,1.0,12.0,694.0,2.0,2023-06-12 15:21:39.111551,Average,Yes,Monthly,House,1483.0
2,2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,Premium,1.0,14.0,NaN,3.0,2023-09-30 15:21:39.221386,Good,Yes,Weekly,House,567.0
3,3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,Basic,1.0,0.0,367.0,1.0,2024-06-12 15:21:39.226954,Poor,Yes,Daily,Apartment,765.0
4,4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,Premium,0.0,8.0,598.0,4.0,2021-12-01 15:21:39.252145,Poor,Yes,Weekly,House,2022.0


In [8]:
# Extracting the Month & Year, then dropping the Policy Start Date column
def date_time_handler(df:pd.DataFrame):
  df['Policy Start Date']=pd.to_datetime(df['Policy Start Date'])
  df['Month']=df['Policy Start Date'].dt.month
  df['Year']=df['Policy Start Date'].dt.year
  df.drop(['Policy Start Date'],axis=1,inplace=True)
  return df

In [9]:
# Applying the date time handler function on train and test dataframe
df_train=date_time_handler(df_train)
df_test=date_time_handler(df_test)

In [10]:
# Making a function which will extract the columns which have missing values below certain threshold
def missing_value_col_below_threshold(df:pd.DataFrame,threshold_value:int)->list:
  
  below_threshold_missing_value=[]

  if not isinstance(threshold_value,int):
    raise TypeError(f"The given value for threshold is {threshold_value}, which is not an integer")
  
  if not isinstance(df,pd.DataFrame):
    raise TypeError(f'The given value of the dataframe is not a pandas dataframe')
  
  rqd_result=(100*df.isnull().sum()/len(df))

  for each_col,each_val in rqd_result.items():
    if each_val<=threshold_value and each_val!=0:
      below_threshold_missing_value.append((each_col,each_val,df[each_col].dtype))

  below_threshold_missing_value=sorted(below_threshold_missing_value,key=lambda x:x[1],reverse=True)

  return below_threshold_missing_value


In [11]:
# Checking if train & test data set has similar columns

missing_train=np.array([each_col for each_col,_,_ in missing_value_col_below_threshold(df_train,50)])
missing_test=np.array([each_col for each_col,_,_ in missing_value_col_below_threshold(df_test,50)])

print(missing_train==missing_test)

[ True  True  True  True  True  True  True  True  True  True  True]


In [12]:
# Also checking if the missing values in these columns are close to each other

missing_train_values=np.array([each_val for _,each_val,_ in missing_value_col_below_threshold(df_train,50)])
missing_test_values=np.array([each_val for _,each_val,_ in missing_value_col_below_threshold(df_test,50)])

print(missing_train_values-missing_test_values)

[-0.0145     -0.05104167  0.05879167 -0.00191667 -0.04916667 -0.008125
  0.01325    -0.002375    0.00208333  0.000125   -0.00016667]


In [13]:
# Checking the columns in train data set which have missing values below or equal to 15 %
missing_value_col_below_threshold(df_train,15)

[('Credit Score', 11.490166666666667, dtype('float64')),
 ('Number of Dependents', 9.139333333333333, dtype('float64')),
 ('Customer Feedback', 6.485333333333333, dtype('O')),
 ('Health Score', 6.173, dtype('float64')),
 ('Annual Income', 3.74575, dtype('float64')),
 ('Age', 1.55875, dtype('float64')),
 ('Marital Status', 1.5440833333333333, dtype('O')),
 ('Vehicle Age', 0.0005, dtype('float64')),
 ('Insurance Duration', 8.333333333333333e-05, dtype('float64'))]

In [14]:
# Making a function to extract the values from the missing_value_col_below_threshold function
# Replacing the missing values in respective columns based upon their data types

def replacing_missing_values(df:pd.DataFrame,missing_value_tuple:list[tuple]):
  result=missing_value_tuple
  col_name,_,data_types =zip(*result)

  for col,data_type in zip(col_name,data_types):
    if data_type=='float64':
      df[col]=df[col].fillna(df[col].median())
    elif data_type=='object':
      df[col]=df[col].fillna(df[col].mode()[0])
  
  return df


In [15]:
# Replacing the missing values in the train dataframe by the median or mode if the missing value percentage is less than or equal to 15
col_to_impute=missing_value_col_below_threshold(df_train,15)
df_train=replacing_missing_values(df_train,col_to_impute)

In [16]:
# Checking the missing value columns
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200000 entries, 0 to 1199999
Data columns (total 22 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   id                    1200000 non-null  int64  
 1   Age                   1200000 non-null  float64
 2   Gender                1200000 non-null  object 
 3   Annual Income         1200000 non-null  float64
 4   Marital Status        1200000 non-null  object 
 5   Number of Dependents  1200000 non-null  float64
 6   Education Level       1200000 non-null  object 
 7   Occupation            841925 non-null   object 
 8   Health Score          1200000 non-null  float64
 9   Location              1200000 non-null  object 
 10  Policy Type           1200000 non-null  object 
 11  Previous Claims       835971 non-null   float64
 12  Vehicle Age           1200000 non-null  float64
 13  Credit Score          1200000 non-null  float64
 14  Insurance Duration    1200000 non-

In [17]:
# Checking the values in the Previous Claims & Occupation columns
print(df_train['Previous Claims'].unique())
print(df_train.Occupation.unique())

[ 2.  1.  0. nan  3.  4.  5.  6.  7.  8.  9.]
['Self-Employed' nan 'Employed' 'Unemployed']


In [18]:
# Returning the columns which are having higher percentage of missing values
col_with_high_missing_values=missing_value_col_below_threshold(df_train,100)

In [19]:
# Function to convert the Occupation column into a numerical variable so that XGBoost classifier will be able to impute missing values
def occupation_converter(occu:str)->int:
  if occu=="Self-Employed":
    return 0
  elif occu=="Employed":
    return 1
  elif occu=="Unemployed":
    return 2
  elif pd.isnull(occu): # np.Nan is special type of float so need to remove it 
    return -1 # To mark missing values
  else:
    return -1 # to handle unexpected values

In [20]:
# Formulating a function to impute the missing values for columns with high percentage of missing values
def preprocessing_high_missing_value_col(df:pd.DataFrame,list_of_tuples:list[tuple])->pd.DataFrame:
  result=list_of_tuples # Taking the list of tuples which contain the columns with high missing values
  high_missing_col,_,_=zip(*result) # Extracting the columns with high missing values
  
  for each_col in high_missing_col:
    if each_col=="Previous Claims":
      df[each_col]=df[each_col].fillna(-1).astype("int64") # Filling missing values with a placeholder of -1 & changing the datatype for XGBClassifer application
    elif each_col=="Occupation":
      df[each_col]=df[each_col].apply(lambda x:occupation_converter(x))
    else:
      pass

  return df

In [21]:
# Making the train dataset ready for the final missing value imputation
df_train=preprocessing_high_missing_value_col(df_train,col_with_high_missing_values)

### Data preparation for the missing value imputation using XGBoost

In [22]:
# Making the suitable datasets
df_prev_col=df_train.drop(columns=['Occupation','Premium Amount','id'],axis=1)
df_occu=df_train.drop(columns=['Previous Claims','Premium Amount','id'],axis=1)

In [23]:
# Checking the first five rows
df_occu.head()

,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,Vehicle Age,Credit Score,Insurance Duration,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Month,Year
0,19.0,Female,10049.0,Married,1.0,Bachelor's,0,22.598761,Urban,Premium,17.0,372.0,5.0,Poor,No,Weekly,House,12,2023
1,39.0,Female,31678.0,Divorced,3.0,Master's,-1,15.569731,Rural,Comprehensive,12.0,694.0,2.0,Average,Yes,Monthly,House,6,2023
2,23.0,Male,25602.0,Divorced,3.0,High School,0,47.177549,Suburban,Premium,14.0,595.0,3.0,Good,Yes,Weekly,House,9,2023
3,21.0,Male,141855.0,Married,2.0,Bachelor's,-1,10.938144,Rural,Basic,0.0,367.0,1.0,Poor,Yes,Daily,Apartment,6,2024
4,21.0,Male,39651.0,Single,1.0,Bachelor's,0,20.376094,Rural,Premium,8.0,598.0,4.0,Poor,Yes,Weekly,House,12,2021


In [24]:
# Extracting the train and test data from occupation columns
df_occu_train=df_occu[df_occu['Occupation']!=-1]
df_occu_test=df_occu[df_occu['Occupation']==-1]

In [25]:
# Extracting the label
y_occu=df_occu_train.pop('Occupation')
X_occu=df_occu_train.copy()

In [26]:
# XGBoost needs all columns to be of numeric type but to handle this with some advanced functionalities, I needed to convert the object columns into categorical columns
X_occu=X_occu.astype({col:'category' for col in X_occu.select_dtypes(include='object').columns})

In [27]:
# Using XGB Classifier
xgb_occu=XGBClassifier(objective='multi:softmax',
                       num_class=y_occu.nunique(),
                       eval_metric='merror',
                       max_depth=5,
                       learning_rate=0.1,
                       n_estimators=100,
                       tree_method='hist',
                       enable_categorical=True)

In [28]:
# Fitting the xgb classifier object on the y_occu & X_occu
xgb_occu.fit(X_occu,y_occu)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=True, eval_metric='merror', feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None, num_class=3,
              num_parallel_tree=None, ...)

In [29]:
# Dropping the occupation column from the df_occu_test
df_occu_test.drop(['Occupation'],axis=1,inplace=True)

In [30]:
# Before predicitng on the df_occu_test, I need to make sure that the object datatype columns are converted into category datatype
df_occu_test=df_occu_test.astype({col:'category' for col in df_occu_test.select_dtypes(include='object').columns})

In [31]:
# Predicting the occupation column by using the xgb_occu 
df_occu_test['Occupation']=xgb_occu.predict(df_occu_test)

In [32]:
# Extracting the occupation column from the df_occu_test
y_occu_test=df_occu_test['Occupation']

In [33]:
# Converting to pandas dataframe & creating new column for merging
y_occu_test=pd.DataFrame(y_occu_test)
y_occu=pd.DataFrame(y_occu)

y_occu_test['INDEX']=y_occu_test.index
y_occu['INDEX']=y_occu.index

In [34]:
# Creating a complete dataframe
occupation_complete_data=pd.merge(y_occu,y_occu_test,on='INDEX',how='outer')

In [35]:
# Changing the NaN to 0, for getting the final column I did the summation
occupation_complete_data=occupation_complete_data.fillna(0).astype('int')
occupation_complete_data['Occupation']=occupation_complete_data['Occupation_x']+occupation_complete_data['Occupation_y']
occupation_complete_data.drop(['Occupation_x','Occupation_y','INDEX'],axis=1,inplace=True)
occupation_complete_data

,Occupation
0,0
1,1
2,0
3,1
4,0
...,...
1199995,2
1199996,0
1199997,2
1199998,1


In [36]:
occupation_complete_data.isnull().sum()

Occupation    0
dtype: int64